In [1]:
%run SAT_help_functions.ipynb

## OUM

In [2]:
data_dir = "/Users/wangjiashun/Desktop/Reserach on SAT/datasets/Lexical Decision Data/SpeedAccData.csv"
sample_size = 16
SAT_conds = [1,2]

In [3]:

class DriftBothSpeedAcc(Drift):
    name = "DriftBothSpeedAcc"
    required_parameters = ["Vspeed","Vacc", "Kspeed", "Kacc"]
    required_conditions = ['SAT']
        
    def get_drift(self, x, t, conditions, **kwargs):
        if conditions['SAT'] == 1:
            return self.Vspeed + x*self.Kspeed 
        elif conditions['SAT'] == 2:
            return self.Vacc + x*self.Kacc
        else: 
            print("Non-implemented SAT conditions")

In [4]:
class BoundSpeedAcc(Bound):
    name = "BoundSpeedAcc"
    required_parameters = ["Bspeed", "Bacc"]
    required_conditions = ['SAT']
    def get_bound(self, conditions, *args, **kwargs):
        assert self.Bacc > 0
        assert self.Bspeed > 0
        if conditions['SAT'] == 1:
            return self.Bspeed
        elif conditions['SAT'] == 2:
            return self.Bacc
        else:
            print("Non-implemented SAT conditions")

In [5]:

class NondecisionSpeedAcc(OverlayNonDecision):
    name = "DriftSpeedAcc"
    required_parameters = ["tSpeed", 'tAcc']
    required_conditions = ['SAT']
        
    def get_nondecision_time(self, conditions, **kwargs):
        if conditions['SAT'] == 1:
            return self.tSpeed 
        elif conditions['SAT'] == 2:
            return self.tAcc
        else: 
            print("Non-implemented SAT conditions")

In [6]:
def get_OUM_result_sub_2v2k2ter(samp, sample_size, v = [0,6], a = [0.5,8], k = [-10,10], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftBothSpeedAcc(Vspeed = Fittable(minval=v[0], maxval=v[1]),
                                              Vacc = Fittable(minval=v[0], maxval=v[1]),
                                              Kspeed=Fittable(minval=k[0], maxval=k[1]),
                                              Kacc=Fittable(minval=k[0], maxval=k[1])), 
                      noise=NoiseConstant(noise=1),
                      bound=BoundConstant(B=Fittable(minval=a[0], maxval=a[1])),                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),

                      ),
                      dx=.01, dt=.01, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,7),
                      columns = model_fit.get_model_parameter_names())
    
    df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'Vspeed': 'Drift Rate Speed',
                            'Vacc': 'Drift Rate Accuracy',
                            'B': 'Boundary Separation',
                            'Kspeed': "Self-excitation Speed",
                            'Kacc': "Self-excitation Accuracy",
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [7]:
participants = [1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]
df_all = pd.DataFrame()
for i in participants:
    for freq in range(1,7):
        print(freq)
        print(i)
        df = pd.read_csv(data_dir)
        df = df[df["Participant"]==i]
        df = df[df["Frequency"]==freq]
        df = df[df["RT"]<5]

        samp = Sample.from_pandas_dataframe(df, rt_column_name="RT", correct_column_name="Correct")
        df_param = get_OUM_result_sub_full(samp)
        df_param['Frequency'] = freq
        df_param["Participant"]==i
        df_all = pd.concat((df_all,df_param))
        print(df_param)
df_OUM = df_all.copy()


1
1


Info: Params [ 2.98912323  3.87178876 -0.35462232  1.6645394   0.50689153  0.5129916
  0.35887806  0.38343689] gave -258.3277872576292


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          2.989123             3.871789              -0.354622   

   Self-excitation Accuracy  Boundary Separation Speed  \
0                  1.664539                   0.506892   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      0.512992                 0.358878   

   Non-decision Time Accuracy         BIC  Frequency  
0                    0.383437 -258.327787          1  
2
1


Info: Params [2.45588652 2.36984766 8.17849471 4.8967868  0.68670988 0.88160731
 0.36295482 0.35793745] gave -134.62489382806234


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          2.455887             2.369848               8.178495   

   Self-excitation Accuracy  Boundary Separation Speed  \
0                  4.896787                    0.68671   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      0.881607                 0.362955   

   Non-decision Time Accuracy         BIC  Frequency  
0                    0.357937 -134.624894          2  
3
1


...

   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0            2.1291             2.655644               4.268533   

   Self-excitation Accuracy  Boundary Separation Speed  \
0                  2.593836                   0.961199   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.932028                 0.306626   

   Non-decision Time Accuracy         BIC  Frequency  
0                    0.338529 -141.307834          4  
5
17


Info: Params [2.46691708 2.29196042 3.73121433 2.73136855 0.83624751 1.5455472
 0.33403101 0.36841448] gave -147.23693845423549


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          2.466917              2.29196               3.731214   

   Self-excitation Accuracy  Boundary Separation Speed  \
0                  2.731369                   0.836248   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.545547                 0.334031   

   Non-decision Time Accuracy         BIC  Frequency  
0                    0.368414 -147.236938          5  
6
17


Info: Params [2.31043727 2.25645465 7.02068949 2.47620944 7.16347652 1.91996466
 0.10489251 0.29689238] gave -66.60982517749227


   Drift Rate Speed  Drift Rate Accuracy  Self-excitation Speed  \
0          2.310437             2.256455               7.020689   

   Self-excitation Accuracy  Boundary Separation Speed  \
0                  2.476209                   7.163477   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.919965                 0.104893   

   Non-decision Time Accuracy        BIC  Frequency  
0                    0.296892 -66.609825          6  


In [20]:
df_OUM.to_csv("OUMfullresultWagenmakers2008.csv")

In [9]:
df_OUM["BIC"].mean()

-75.24333849573985

In [11]:
df_OUM

,Drift Rate Speed,Drift Rate Accuracy,Self-excitation Speed,Self-excitation Accuracy,Boundary Separation Speed,Boundary Separation Accuracy,Non-decision Time Speed,Non-decision Time Accuracy,BIC,Frequency
0,2.989123,3.871789,-0.354622,1.664539,0.506892,0.512992,0.358878,0.383437,-258.327787,1
0,2.455887,2.369848,8.178495,4.896787,0.686710,0.881607,0.362955,0.357937,-134.624894,2
0,0.904519,1.749850,4.318621,4.821636,0.612835,0.716453,0.349055,0.386201,-12.876078,3
0,2.377517,2.863789,0.374389,6.606345,0.503802,0.687986,0.342263,0.381301,-161.423393,4
0,3.537550,2.963319,6.827641,7.700286,1.217458,1.757385,0.317570,0.304870,-174.388412,5
...,...,...,...,...,...,...,...,...,...,...
0,2.777299,2.469134,9.840515,2.312019,6.639324,1.374513,0.195933,0.362363,-210.092219,2
0,1.823524,1.914526,6.531255,3.246480,1.069556,1.386540,0.301128,0.389924,-11.019406,3
0,2.129100,2.655644,4.268533,2.593836,0.961199,1.932028,0.306626,0.338529,-141.307834,4
0,2.466917,2.291960,3.731214,2.731369,0.836248,1.545547,0.334031,0.368414,-147.236938,5


In [18]:
p = np.array([[i]*6 for i in participants]).flatten()

In [19]:
df_OUM['Participant'] = p